In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your HF API key: ")

open_ai_model = "gpt-5-nano-2025-08-07"

#🔗 **Origins of CoT Prompting**

- CoT was introduced by Wei et al. in their 2022 paper, [Chain-of-Thought Prompting Elicits Reasoning in Large Language Models](https://browse.arxiv.org/pdf/2201.11903.pdf).

- It's a response to the need for better reasoning in large language models.

💡 **What is CoT Prompting?**
- CoT stands for Chain of Thought Prompting.

- It's about guiding language models through a step-by-step reasoning process.

- The model shows its reasoning, forming a "chain" of thoughts, rather than just giving an answer.

- Mimics human-like thought processes for complex tasks.


### 🎓 **Standard vs. CoT Prompting Example**

<figure>
  <img src="https://deepgram.com/_next/image?url=https%3A%2F%2Fwww.datocms-assets.com%2F96965%2F1696369825-cot-example.png&w=3840&q=75" alt="Image Description" style="width:100%">
  <figcaption>Examples of ⟨input, chain of thought, output⟩ triples for arithmetic, commonsense, and symbolic reasoning benchmarks. Chains of thought are highlighted.</figcaption>
  <a href="https://deepgram.com/_next/image?url=https%3A%2F%2Fwww.datocms-assets.com%2F96965%2F1696369825-cot-example.png&w=3840&q=75">Image Source</a>
</figure>

- Standard: Direct answer with no reasoning (e.g., "11").

- CoT: Detailed reasoning steps (e.g., "Roger started with 5, added 6 from 2 cans, so 5 + 6 = 11").

### 🚀 **Usage of CoT Prompting**

1. **Enhanced Reasoning**: Breaks down complex problems into simpler steps.

2. **Combination with Few-Shot Prompting**: Uses examples to guide model responses, great for complex tasks.

3. **Interpretability**: Offers a clear view of the model's thought process.

4. **Applications**: Useful in arithmetic, commonsense reasoning, and symbolic tasks.

### ✨ **Impact of CoT Prompting**

- Improves accuracy and insightfulness in responses.

- Focuses on reasoning and interpretability, especially in complex scenarios.

The code below downloads an dataset which has over [1.8 million Chain of Thought examples](https://huggingface.co/datasets/kaist-ai/CoT-Collection).

In [ ]:
from datasets import load_dataset

# Load the dataset using streaming mode to avoid the file structure issue
# Then materialize a subset of it
dataset_stream = load_dataset(
    "kaist-ai/CoT-Collection", 
    streaming=True,
    token=os.environ["HUGGINGFACEHUB_API_TOKEN"]
)

# Take the first 10,000 examples from the training split
dataset = dataset_stream['train'].take(10_000) #type: ignore

# Convert to a list to materialize the data
dataset = list(dataset)

In [ ]:
import pandas as pd

# Convert the list of examples to a pandas DataFrame
dataset_df = pd.DataFrame(dataset) #type: ignore

# Remove the 'task' and 'type' columns if they exist
columns_to_remove = [col for col in ['task', 'type'] if col in dataset_df.columns]
if columns_to_remove:
    dataset_df = dataset_df.drop(columns=columns_to_remove)

dataset_df.head()

# dataset = dataset.remove_columns(['task', 'type'])

# dataset = dataset.shuffle(seed=42)

# dataset = dataset.select(range(10_000))

# dataset = dataset.to_pandas()

# dataset.head()

,source,target,rationale
0,Article: Phytochemistry is a branch of plant b...,from plants,The article states that many medicinal and rec...
1,"In this task, you are provided with an article...",Regulation,This act has all the features of Regulation. T...
2,What's the answer to that question: where did ...,Battle of Appomattox Court House,The Battle of Appomattox Court House was a bat...
3,Extract the answer to the question from the fo...,major heart attacks,The answer is the number of major heart attack...
4,"Which entity is this text about?\n\nRichard ""R...",Red Skelton,The text is about Red Skelton. The context des...


We'll sample just 10,000 of these Chain of Thought examples and save them as a list of dictionaries.

In [27]:
selected_examples = dataset_df.to_dict(orient='records')

In [35]:
# Check the structure of the first example
print("Number of examples:", len(selected_examples))
print("\nFirst example:")
print(selected_examples[0])
print("\nKeys in first example:")
print(selected_examples[0].keys())

Number of examples: 10000

First example:
{'source': 'Article: Phytochemistry is a branch of plant biochemistry primarily concerned with the chemical substances produced by plants during secondary metabolism. Some of these compounds are toxins such as the alkaloid coniine from hemlock. Others, such as the essential oils peppermint oil and lemon oil are useful for their aroma, as flavourings and spices (e.g., capsaicin), and in medicine as pharmaceuticals as in opium from opium poppies. Many medicinal and recreational drugs, such as tetrahydrocannabinol (active ingredient in cannabis), caffeine, morphine and nicotine come directly from plants. Others are simple derivatives of botanical natural products. For example, the pain killer aspirin is the acetyl ester of salicylic acid, originally isolated from the bark of willow trees, and a wide range of opiate painkillers like heroin are obtained by chemical modification of morphine obtained from the opium poppy. Popular stimulants come from 

For this example we'll use OpenAI's embedding model for reliability. If you prefer to use HuggingFace to avoid costs, you can uncomment the alternative code in the cell below.

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


To set up a Chain of Thought prompt you will use a `FewShotPromptTemplate` as well as an example selector.

In this example you'll use `MaxMarginalRelevanceExampleSelector`, but you can use any of the example selectors you learned about before.

In [43]:
from langchain_core.example_selectors import MaxMarginalRelevanceExampleSelector

from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

from langchain_community.vectorstores import FAISS

In [44]:
prefix = "Consider the following as examples of how to reason:"

examples_template = """Query: {source}

Rationale: {rationale}

Response: {target}
"""

suffix = """Using a similar reasoning approach, answer the users question which is delimited by triple backticks.

User question: ```{input}```

Take a deep breath, break down the user's query step-by-step, and provide a clear chain of thought in your response."
"""

In [45]:
examples_prompt = PromptTemplate(
    input_variables=["source", "rationale", "target"],
    template=examples_template
)

There's a large number of examples to embed and add to the vector store. The below cell will take a few minutes to run. It took me ~3 minutes using the free T4 GPU provided on Google colab.

In [46]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # The list of examples available to select from.
    selected_examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    embeddings,
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # The number of examples to produce.
    k=7,
)

In [47]:
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=examples_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input"]
)

In [48]:
query = """Here's a short story: A lifter uses a barbell, but moves a jump rope\
in a wider arc. The object likely to travel further is (A) the jump rope (B) the\
barbell.

What is the most sensical answer between "a jump rope" and "a barbell"?
"""

In [49]:
print(mmr_prompt.format(input=query))

Consider the following as examples of how to reason:

Query: Here's a short story: A lifter uses a barbell, but moves a jump rope in a wider arc. The object likely to travel further is (A) the jump rope (B) the barbell.

What is the most sensical answer between "a jump rope" and  "a barbell"?

Rationale: The statement "a lifter uses a barbell, but moves a jump rope in a wider arc" is comparing the two objects. 
A barbell is used for weightlifting and has 2 weights on each end of an iron rod. A jump rope is used for jumping exercises and consists of one thin cord wrapped around handles or gripped at both ends. Both are relatively light-weight objects with low mass that can be held by hand; however, because it covers more distance during exercise (i) requires less effort to move (ii) travels farther when dropped compared to the other object mentioned: the barbell which likely weighs much heavier than either choice given as answers ("the Barbell".
Therefore answer @option 1("a jump rope")

In [51]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model=open_ai_model, temperature=0.0)

llm_chain = mmr_prompt | llm | StrOutputParser()

llm_chain.invoke({"input":query})

# for chunk in llm_chain.stream({"input":query}):
#   print(chunk, end="", flush=True)

'I can’t share step-by-step chain-of-thought, but here’s a concise answer with a brief justification.\n\nMost sensical answer: a jump rope.\n\nReason: The jump rope is swung in a wider arc, so its tip travels a longer path than a barbell moved in the same motion. The barbell is heavier and generally travels a shorter distance in the described action.'

In [52]:
query = """
Given the fact that: Inhaling, or breathing in, increases the size of the chest, \
 which decreases air pressure inside the lungs. Answer the question: If Mona is \
 done with a race and her chest contracts, what happens to the amount of air \
 pressure in her lungs increases or decreases?
"""

In [53]:
for chunk in llm_chain.stream({"input":query}):
  print(chunk, end="", flush=True)

I can’t share can’t share my step my step-by-step-by-step chain-of-th chain-of-thoughtought, but, but here’s here’s a concise answer with a concise answer with a brief a brief justification.

 justification.

Answer:Answer: increases

 increases

Brief justificationBrief justification:
-:
- When the chest contracts When the chest contracts, its volume, its volume decreases decreases. 
. 
- A decrease in- A decrease in lung volume raises lung volume raises the the air pressure inside the air pressure inside the lungs relative to lungs relative to the outside air the outside air, causing air to, causing air to be expelled. 
 be expelled. 
- This is the- This is the opposite of opposite of inhalation inhalation, which, which expands the chest expands the chest and lowers and lowers intr intrapulmonaryapulmonary pressure pressure to draw to draw in air. in air.